<a href="https://colab.research.google.com/github/MattJCR/Red-neuronal-Basica/blob/master/Red_neuronal_Basica.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import numpy as np
from sklearn import datasets
from sklearn.preprocessing import normalize
from sklearn.model_selection import train_test_split
from keras.utils import np_utils

## Modificación de la implementación de la red neuronal

* Agrega el método weigths() a la clase Network, de tal forma que permita obtener los pesos de las neuronas

* Agrega el método set_weights() a la clase Network, de tal forma que permite definir los pesos de las neuronas
     

* Agrega los métodos sigmoide() y sigmoide_prime() a la clase Network

* Modifica la clase Network, para que se pueda decidir qué función de activación utilizar: relu() o sigmoide()
     

* Los métodos predict() y accuracy() de la clase Network están implementados para resolver problemas de clasificación

* Modifícalos de tal manera que también se puedan utilizar con problemas de regresión
     

* Modifica el método backpropagation() de tal manera que devuelva como resultado el array de valores de los nodos durante las épocas de entrenamiento
     

* Una vez implementados los cambios, entrena la red neuronal del ejemplo de los apuntes
```
examples = []
examples.append([[0.5, 0.67, 0.5], [0.25, 0.6]])
```

* Ejecuta la red neuronal para los datos de ejemplo de los apuntes

* Comprueba los valores de los nodos y de los pesos

* Los valores de los nodos tienen que ser los mismos que los de los apuntes

* Los valores de los pesos son ligeramente diferentes, ¿por qué?

```
net = Network([3, 4, 2])

net.set_weights([0.1, 0.2, 0.3, 0.4, 0.5, 0.6])
```

o

```
net.set_weights([[0.1, 0.1, 0.1], [0.2, 0.2, 0.2], [0.3, 0.3, 0.3], [0.4, 0.4, 0.4], [0.5, 0.5, 0.5, 0.5], [0.6, 0.6, 0.6, 0.6]])

valores_nodos = net.backpropagation(0.9, examples, 1)

print(valores_nodos)
net.weights()
```

In [ ]:
class NetNode(object):
  def __init__(self):
    self.inputs = []
    self.weights = []
    self.value = None

class Network(object):

  def __init__(self, layers, function = 'relu', prediction_type = 'classification'):
    # regression
    self.net = [[NetNode() for _ in range(size)] for size in layers]
    self.function = function
    self.prediction_type = prediction_type
    sizes = len(layers)

    for layer in range(1,sizes):
      for node in self.net[layer]:
        for unit in self.net[layer - 1]:
          node.inputs.append(unit)
          node.weights.append(0)
        node.weights = [np.random.uniform() for _ in range(len(node.weights))]

  def backpropagation(self, eta, examples, epochs):
        inputs = self.net[0]
        outputs = self.net[-1]
        layer_size = len(self.net)
        '''
        for layer in self.net[1:]:
            for node in layer:
                node.weights = [np.random.uniform() for _ in range(len(node.weights))]
        '''
        for epoch in range(epochs):
            for x_train, y_train in examples:
                for value, node in zip(x_train, inputs):
                    node.value = value
                for layer in self.net[1:]:
                    for node in layer:
                        in_val = [n.value for n in node.inputs]
                        unit_value = np.dot(in_val, node.weights)
                        if self.function == 'sigmoid':
                          node.value = self.sigmoide(unit_value)
                        else:
                          node.value = self.relu(unit_value)
                delta = [[] for _ in range(layer_size)]
                err = [y_train[i] - outputs[i].value for i in range(len(outputs))]
                if self.function == 'sigmoid':
                  delta[-1] = [self.sigmoide_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
                else:
                  delta[-1] = [self.relu_prime(outputs[i].value) * err[i] for i in range(len(outputs))]
                hidden_layers = layer_size - 2
                for i in range(hidden_layers, 0, -1):
                    layer = self.net[i]
                    n_layers = len(layer)
                    w = [[node.weights[l] for node in self.net[i + 1]] for l in range(n_layers)]
                    if self.function == 'sigmoid':
                      delta[i] = [self.sigmoide_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
                    else:
                      delta[i] = [self.relu_prime(layer[j].value) * np.dot(w[j], delta[i + 1]) for j in range(n_layers)]
                for i in range(1, layer_size):
                    layer = self.net[i]
                    in_val = [node.value for node in self.net[i - 1]]
                    n_layers = len(self.net[i])
                    for j in range(n_layers):
                        layer[j].weights = np.add(layer[j].weights, np.multiply(eta * delta[i][j], in_val))
            # print(f"epoch {epoch}/{epochs} | total error={np.sum(err)/len(examples)}")
    
  def predict(self,input_data):
    inputs = self.net[0]

    for v, n in zip(input_data, inputs):
      n.value = v

    for layer in self.net[1:]:
      for node in layer:
        in_val = [n.value for n in node.inputs]
        unit_value = np.dot(in_val, node.weights)
        if self.function == 'sigmoid':
          node.value = self.sigmoide(unit_value)
        else:
          node.value = self.relu(unit_value)
    
    outputs = self.net[-1]
    if self.prediction_type == 'regression':
      return outputs
    else:
      return outputs.index(max(outputs, key=lambda node: node.value))
  def accuracy(self, examples):
    correct = 0
    for x_test, y_test in examples:
      prediction = self.predict(x_test)

      if(y_test[prediction] == 1):
        correct += 1
    return correct / len(examples)

  def relu(self,z):
    return max(0,z)

  def relu_prime(self, z):
    return 1 if z > 0 else 0

  def sigmoide(self, z):
    return 1/(1 + np.exp(-z))

  def sigmoide_prime(self, z):
    return z * (1 - z)
  
  def set_weights(self, layers):
    try:
      index = 0
      for layer in self.net:
        for node in layer:
          node.weights = layers[index]
          index += 1
    except:
      print("An exception occurred")

  def weights(self):
    weights = []
    if self.net != None:
      for layer in self.net:
        for node in layer:
          weights.append(node.weights)
    return weights
  

In [ ]:
iris_X,iris_y = datasets.load_iris(return_X_y = True)

In [ ]:
iris_X[:10]

array([[5.1, 3.5, 1.4, 0.2],
       [4.9, 3. , 1.4, 0.2],
       [4.7, 3.2, 1.3, 0.2],
       [4.6, 3.1, 1.5, 0.2],
       [5. , 3.6, 1.4, 0.2],
       [5.4, 3.9, 1.7, 0.4],
       [4.6, 3.4, 1.4, 0.3],
       [5. , 3.4, 1.5, 0.2],
       [4.4, 2.9, 1.4, 0.2],
       [4.9, 3.1, 1.5, 0.1]])

In [ ]:
iris_y[:10]

array([0, 0, 0, 0, 0, 0, 0, 0, 0, 0])

In [ ]:
iris_x_normalized = normalize(iris_X,axis=0)
iris_x_normalized[:10]

array([[0.07056264, 0.09254209, 0.02754801, 0.01150242],
       [0.06779548, 0.07932179, 0.02754801, 0.01150242],
       [0.06502832, 0.08460991, 0.02558029, 0.01150242],
       [0.06364474, 0.08196585, 0.02951572, 0.01150242],
       [0.06917906, 0.09518615, 0.02754801, 0.01150242],
       [0.07471338, 0.10311833, 0.03345115, 0.02300485],
       [0.06364474, 0.08989803, 0.02754801, 0.01725364],
       [0.06917906, 0.08989803, 0.02951572, 0.01150242],
       [0.06087757, 0.07667773, 0.02754801, 0.01150242],
       [0.06779548, 0.08196585, 0.02951572, 0.00575121]])

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(iris_x_normalized,iris_y,test_size=0.2, shuffle=True)

In [ ]:
y_train = np_utils.to_categorical(y_train,num_classes=3)
y_test = np_utils.to_categorical(y_test,num_classes=3)

In [ ]:
examples = []
for i in range(len(X_train)):
  examples.append([X_train[i],y_train[i]])

In [ ]:
net = Network([4,7,3])

In [ ]:
net.weights()

[[],
 [],
 [],
 [],
 [0.2819148059916814,
  0.4762099994579905,
  0.03891883892502079,
  0.8643636294309746],
 [0.5852459149057281,
  0.3018587450045592,
  0.5423861654492995,
  0.495178596795549],
 [0.30657533373434653,
  0.0016214139640438052,
  0.8389777738236476,
  0.7408010274358283],
 [0.7761926749624963,
  0.07179016565757701,
  0.3472570645124383,
  0.57544773570609],
 [0.7398565081582099,
  0.1947228195252544,
  0.1270063110469215,
  0.8119759304795575],
 [0.6407442103975245,
  0.6586297228750108,
  0.5669002923472392,
  0.23620789464166025],
 [0.7195395258884627,
  0.5792360111260989,
  0.9952352694921137,
  0.8311453993498296],
 [0.376585622962921,
  0.3239979987476642,
  0.17458414167730318,
  0.17216017126740646,
  0.24874311318199493,
  0.9968088341676081,
  0.24284946076675817],
 [0.48640867657489095,
  0.5791384854279996,
  0.6775169374494869,
  0.7144615212322427,
  0.7706174506951905,
  0.9164340441888907,
  0.2717327353718537],
 [0.3403967847083579,
  0.8056988709671

In [ ]:
layers_weights = [
    # Layer 0
    [],
    [],
    [],
    [],
    # Layer 1
    [ 0.1, -0.1 ,  0.1,  1.1 ],
    [0.1, 0.1, 1.1, 1.1],
    [ 0.1,  0.1, -0.1,  0.1 ],
    [-0.1, -0.1, -0.1,  0.1],
    [ 0.1,  1.1,  0.1, -1.1],
    [ 1.1,  1.1, -0.1, -1.1],
    [ 0.1,  1.1, -0.1,  0.1],
    # Layer 2
    [-0.1 , -0.1,  0.1,  0.1,  1.1, 2.1,  0.1],
    [ 0.1,  0.1,  0.1,  0.1,  0.1, 0.1 , -0.1],
    [ 1.1,  0.1,  0.1,  0.1, -0.1, -0.1, -0.1]
]
net.set_weights(layers_weights)
net.weights()

[[],
 [],
 [],
 [],
 [0.1, -0.1, 0.1, 1.1],
 [0.1, 0.1, 1.1, 1.1],
 [0.1, 0.1, -0.1, 0.1],
 [-0.1, -0.1, -0.1, 0.1],
 [0.1, 1.1, 0.1, -1.1],
 [1.1, 1.1, -0.1, -1.1],
 [0.1, 1.1, -0.1, 0.1],
 [-0.1, -0.1, 0.1, 0.1, 1.1, 2.1, 0.1],
 [0.1, 0.1, 0.1, 0.1, 0.1, 0.1, -0.1],
 [1.1, 0.1, 0.1, 0.1, -0.1, -0.1, -0.1]]

In [ ]:
net.backpropagation(0.03,examples,100)

In [ ]:
examples = []
for i in range(len(X_test)):
  examples.append([X_test[i],y_test[i]])

In [ ]:
prediction = net.predict(X_test[0])
accuracy = net.accuracy(examples)

print(f"Accuracy:  {accuracy}")
print(f"Desired output:  {y_test[0]}")
print(f"Index of output:  {prediction}")

Accuracy:  0.9666666666666667
Desired output:  [1. 0. 0.]
Index of output:  0


In [ ]:
net.weights()

[[],
 [],
 [],
 [],
 array([-0.13283458, -0.69871849,  0.8305343 ,  2.14311208]),
 array([ 0.22411461, -0.35561525,  1.81931643,  1.53291508]),
 array([ 0.13283531,  0.05825461, -0.03363811,  0.10037829]),
 array([-0.1, -0.1, -0.1,  0.1]),
 array([ 0.71034223,  1.84699377, -0.03060711, -1.56661056]),
 array([ 1.90998521,  2.05377802, -0.44504792, -2.08865112]),
 array([ 0.18829112,  1.2475826 , -0.22585293, -0.11703413]),
 array([-1.00863722, -1.26818609,  0.06130895,  0.1       ,  1.55979124,
         2.72541317,  0.18509538]),
 array([ 0.26418472,  1.00079817,  0.12904298,  0.1       ,  0.13802821,
         0.55639755, -0.17872238]),
 array([ 2.18796712,  1.03226658,  0.06000817,  0.1       , -1.63963538,
        -2.32555092, -0.6340222 ])]

In [ ]:
#@title Prueba con sigmoid

net = Network([4,7,3],function = 'sigmoid', prediction_type = 'regression')

examples = []
for i in range(len(X_train)):
  examples.append([X_train[i],y_train[i]])

net.backpropagation(0.03,examples,100)

examples = []
for i in range(len(X_test)):
  examples.append([X_test[i],y_test[i]])

prediction = net.predict(X_test[0])
print(prediction)
accuracy = net.accuracy(examples)

print(f"Accuracy:  {accuracy}")
print(f"Desired output:  {y_test[0]}")
print(f"Index of output:  {prediction}")

[<__main__.NetNode object at 0x7fc3f972f880>, <__main__.NetNode object at 0x7fc3f96f3d00>, <__main__.NetNode object at 0x7fc3f96f3ee0>]


IndexError: ignored